In [14]:
'''
Author: xushengyichn 54436848+xushengyichn@users.noreply.github.com
Date: 2023-01-24 14:28:48
LastEditors: xushengyichn 54436848+xushengyichn@users.noreply.github.com
LastEditTime: 2023-01-25 18:51:17
FilePath: \20230124优化问题\bayesian.ipynb
Description: 创建贝叶斯优化

Copyright (c) 2023 by xushengyichn 54436848+xushengyichn@users.noreply.github.com, All Rights Reserved. 
'''


'\nAuthor: xushengyichn 54436848+xushengyichn@users.noreply.github.com\nDate: 2023-01-24 14:28:48\nLastEditors: xushengyichn 54436848+xushengyichn@users.noreply.github.com\nLastEditTime: 2023-01-25 18:51:17\nFilePath: \x8230124优化问题\x08ayesian.ipynb\nDescription: 创建贝叶斯优化\n\nCopyright (c) 2023 by xushengyichn 54436848+xushengyichn@users.noreply.github.com, All Rights Reserved. \n'

In [15]:
#%% 1. 导入库
import time
import matlab
import matlab.engine
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import cross_val_score,KFold,cross_validate
from sklearn.linear_model import LogisticRegression

In [16]:
#%% 2. 定义函数
eng = matlab.engine.start_matlab()
def black_box_function(mTMD1,mTMD2,fTMD1,fTMD2,fTMD3,dTMD1,dTMD2,dTMD3,xTMD1,xTMD2,xTMD3):
    total_tmd_mass_ratio = 0.02 # 总质量比 The total mass ratio
    mass_six_span = 10007779.7 # 深中通道非通航桥六跨连续梁质量 The mass of 6-span continuous beam of the non-navigational bridge of the Zhenzhong-Link
    total_tmd_mass = total_tmd_mass_ratio * mass_six_span # 总质量 The total mass
    
    mTMD1=0.01*total_tmd_mass+mTMD1*total_tmd_mass*0.49 # 质量 The mass mTMD1
    mTMD2=0.01*total_tmd_mass+mTMD2*total_tmd_mass*0.49 # 质量 The mass mTMD2
    mTMD3=total_tmd_mass-mTMD1-mTMD2 # 质量 The mass mTMD3
    fTMD1=0.7+fTMD1*0.3 # 频率 The frequency fTMD1
    fTMD2=0.7+fTMD2*0.3 # 频率 The frequency fTMD2
    fTMD3=0.7+fTMD3*0.3 # 频率 The frequency fTMD3
    dTMD1=0.02+dTMD1*0.18 # 阻尼比 The damping ratio dTMD1
    dTMD2=0.02+dTMD2*0.18 # 阻尼比 The damping ratio dTMD2
    dTMD3=0.02+dTMD3*0.18 # 阻尼比 The damping ratio dTMD3
    xTMD1=xTMD1*660 # TMD1的x坐标 The x-coordinate of TMD1
    xTMD2=xTMD2*660 # TMD2的x坐标 The x-coordinate of TMD2
    xTMD3=xTMD3*660 # TMD3的x坐标 The x-coordinate of TMD3
    t_length=matlab.double(100) # 时间长度 The time length
    number_of_modes_to_control=matlab.double([1,2,3,4,5,6]) # 控制模态 The controlled modes
    number_of_modes_to_consider=10 # 考虑模态 The considered modes
    number_of_tmds=3 # TMD数量 The number of TMDs
    modal_damping_ratios=np.ones((1,number_of_modes_to_consider))*0.003 # 模态阻尼比 The modal damping ratios
    
    result = -eng.b_0_3_tmd(number_of_modes_to_control,number_of_modes_to_consider,number_of_tmds,modal_damping_ratios,t_length,mTMD1,mTMD2,fTMD1,fTMD2,fTMD3,dTMD1,dTMD2,dTMD3,xTMD1,xTMD2,xTMD3,total_tmd_mass)
    return result

In [17]:
#%% 3. 定义优化器
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={"mTMD1": (0,1), 
             "mTMD2": (0,1),
             "fTMD1": (0,1),
             "fTMD2": (0,1),
             "fTMD3": (0,1),
             "dTMD1": (0,1),
             "dTMD2": (0,1),
             "dTMD3": (0,1),
             "xTMD1": (0,1),
             "xTMD2": (0,1),
             "xTMD3": (0,1),
             },
    verbose=1,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

optimizer.maximize(
        init_points=10,
        n_iter=100,
        acq="ei",  # Expected Improvement.
        # kappa=7.2088954429132,
        xi=0.0012265019946638392,
        # What follows are GP regressor parameters
        kernel=Matern(nu=1.5),
        # alpha=1,
        # alpha=2.2492215663634233e-10,
        alpha=8.960456511761172e-06,
        normalize_y=True,
        n_restarts_optimizer=5,
)

|   iter    |  target   |   dTMD1   |   dTMD2   |   dTMD3   |   fTMD1   |   fTMD2   |   fTMD3   |   mTMD1   |   mTMD2   |   xTMD1   |   xTMD2   |   xTMD3   |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
| 2         | -0.8711   | 0.6852    | 0.2045    | 0.8781    | 0.02739   | 0.6705    | 0.4173    | 0.5587    | 0.1404    | 0.1981    | 0.8007    | 0.9683    |
| 3         | -0.76     | 0.3134    | 0.6923    | 0.8764    | 0.8946    | 0.08504   | 0.03905   | 0.1698    | 0.8781    | 0.09835   | 0.4211    | 0.9579    |
| 4         | -0.6844   | 0.5332    | 0.6919    | 0.3155    | 0.6865    | 0.8346    | 0.01829   | 0.7501    | 0.9889    | 0.7482    | 0.2804    | 0.7893    |
| 8         | -0.6552   | 0.3478    | 0.7508    | 0.726     | 0.8833    | 0.6237    | 0.7509    | 0.3489    | 0.2699    | 0.8959    | 0.4281    | 0.9648    |
| 20        | -0.6101   | 0.2896    | 0.8197    | 0.